In [1]:
import os
import re

import pandas as pd

from dotenv import load_dotenv
from bs4 import BeautifulSoup

from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEndpoint
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_transformers import MarkdownifyTransformer
from langchain_core.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableMap, RunnableBranch
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import LLMChain, RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.example_selectors import SemanticSimilarityExampleSelector

/opt/conda/envs/raghr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 환경변수 설정

# metadata 출력함수 템플릿 정의

In [4]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )


# 문서로딩

## csv

In [16]:
file_path = os.path.join(os.path.dirname(os.getcwd()), 
                                          'data/files/[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv')
lecture_df = pd.read_csv(file_path, skiprows = 4)
print(lecture_df.head())  # 데이터 미리보기
print(lecture_df.columns)  # 컬럼 확인

lecture_df = lecture_df.drop(['Unnamed: 0'], axis = 1)

   Unnamed: 0                              관련 모듈              강의 목록  \
0         NaN  프로젝트 수행을 위한 이론 1 : Python\n\n(기초)  Part1 파이썬 기본기 다지기   
1         NaN                                NaN  Part1 파이썬 기본기 다지기   
2         NaN                                NaN  Part1 파이썬 기본기 다지기   
3         NaN                                NaN  Part1 파이썬 기본기 다지기   
4         NaN                                NaN  Part1 파이썬 기본기 다지기   

                  파트명                                                클립명  \
0  Ch01. 파이썬에 대한 모든 것               Intro. Selena 강좌를 듣기 전 필독! 학습 방법 5단계   
1  Ch01. 파이썬에 대한 모든 것                                  CH01-01. 파이썬 활용분야   
2       Ch02. 변수와 자료형  CH02-01. 변수(variable), 자료형(data type), 숫자형(num...   
3       Ch02. 변수와 자료형                         CH02-02. 문자열 자료형(string) I   
4       Ch02. 변수와 자료형                        CH02-03. 문자열 자료형(string) II   

     클립 시간      수강 일자 일자별 수강 시간   수강 완료 일자  
0  0:10:52  11/14 (목)   4:07:46  11/26 (화)  
1  0:09:44        NaN     

In [17]:
lecture_df

,관련 모듈,강의 목록,파트명,클립명,클립 시간,수강 일자,일자별 수강 시간,수강 완료 일자
0,프로젝트 수행을 위한 이론 1 : Python\n\n(기초),Part1 파이썬 기본기 다지기,Ch01. 파이썬에 대한 모든 것,Intro. Selena 강좌를 듣기 전 필독! 학습 방법 5단계,0:10:52,11/14 (목),4:07:46,11/26 (화)
1,NaN,Part1 파이썬 기본기 다지기,Ch01. 파이썬에 대한 모든 것,CH01-01. 파이썬 활용분야,0:09:44,NaN,NaN,NaN
2,NaN,Part1 파이썬 기본기 다지기,Ch02. 변수와 자료형,"CH02-01. 변수(variable), 자료형(data type), 숫자형(num...",0:13:50,NaN,NaN,NaN
3,NaN,Part1 파이썬 기본기 다지기,Ch02. 변수와 자료형,CH02-02. 문자열 자료형(string) I,0:13:48,NaN,NaN,NaN
4,NaN,Part1 파이썬 기본기 다지기,Ch02. 변수와 자료형,CH02-03. 문자열 자료형(string) II,0:10:21,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
679,NaN,Anomaly Detection,anomaly detection 대회 case study,Anomaly Dectection Case study Overview,0:11:44,NaN,NaN,NaN
680,NaN,Anomaly Detection,anomaly detection 대회 case study,Anomaly Dectection Case study - Case1,0:18:15,NaN,NaN,NaN
681,NaN,Anomaly Detection,anomaly detection 대회 case study,Anomaly Dectection Case study - Case2,0:20:47,NaN,NaN,NaN
682,NaN,Anomaly Detection 대회,대회 안내,대회 소개,0:09:09,NaN,NaN,NaN


In [18]:
# 전처리함수 / 병합 셀 fill
def fill_between_non_nan(df, column):
    filled_values = []
    temp_value = None  # 마지막으로 본 NaN이 아닌 값 저장
    
    for value in df[column]:
        if pd.notna(value):  # NaN이 아닌 값이 나타나면 저장
            temp_value = value
        filled_values.append(temp_value)  # 현재 값 또는 가장 최근 값을 추가
    
    df[column] = filled_values  # 결과 반영
    
    return df

In [19]:
# csv
for i in lecture_df.columns:
    lecture_df2 = fill_between_non_nan(lecture_df, i)

lecture_df2.to_csv(os.path.join(os.path.dirname(os.getcwd()), 
                               'data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv'))
# skiprows = 4 적용한 CSV 로더 생성
loader = CSVLoader(
    file_path=os.path.join(os.path.dirname(os.getcwd()),
                           'data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv'),
                           csv_args={
                               'delimiter': ',',
                               }
                               )

# 데이터 로드
csv_docs = loader.load()

print(len(csv_docs)) #행의갯수
#print(docs[0])

for doc in csv_docs:
    doc.metadata["domain"] = "lecture_info"  # 강의 정보 라벨링
    print(doc)

684
page_content=': 0
관련 모듈: 프로젝트 수행을 위한 이론 1 : Python

(기초)
강의 목록: Part1 파이썬 기본기 다지기
파트명: Ch01. 파이썬에 대한 모든 것
클립명: Intro. Selena 강좌를 듣기 전 필독! 학습 방법 5단계
클립 시간: 0:10:52
수강 일자: 11/14 (목)
일자별 수강 시간: 4:07:46
수강 완료 일자: 11/26 (화)' metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv', 'row': 0, 'domain': 'lecture_info'}
page_content=': 1
관련 모듈: 프로젝트 수행을 위한 이론 1 : Python

(기초)
강의 목록: Part1 파이썬 기본기 다지기
파트명: Ch01. 파이썬에 대한 모든 것
클립명: CH01-01. 파이썬 활용분야
클립 시간: 0:09:44
수강 일자: 11/14 (목)
일자별 수강 시간: 4:07:46
수강 완료 일자: 11/26 (화)' metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv', 'row': 1, 'domain': 'lecture_info'}
page_content=': 2
관련 모듈: 프로젝트 수행을 위한 이론 1 : Python

(기초)
강의 목록: Part1 파이썬 기본기 다지기
파트명: Ch02. 변수와 자료형
클립명: CH02-01. 변수(variable), 자료형(data type), 숫자형(number)
클립 시간: 0:13:50
수강 일자: 11/14 (목)
일자별 수강 시간: 4:07:46
수강 완료

## html

In [20]:
html_file_path = os.path.join(os.path.dirname(os.getcwd()), "data/files/page.html")
save_html_to_text = os.path.join(os.path.dirname(os.getcwd()), "data/files/page.txt")

# 1️⃣ HTML 파일 직접 열기
with open(html_file_path, "r", encoding="utf-8") as f:
    html_content = f.read()

# 2️⃣ BeautifulSoup으로 HTML 파싱
soup = BeautifulSoup(html_content, "html.parser")
#soup
with open(save_html_to_text, "w", encoding="utf-8") as f:
    f.write(soup.prettify())

In [21]:
# 텍스트 로더 생성
loader = TextLoader("/home/data/Langchain/qa_engine_test/data/files/page.txt")

# 문서 로드
docs = loader.load()

In [22]:
md = MarkdownifyTransformer()
converted_docs = md.transform_documents(docs)

In [23]:
def preprocess_text(text):
    # Example: Remove all HTML tags
    text = re.sub(r"!\[.*?\]\(.*?\)", "", text) 
    text = re.sub(r'\n\n\d+(\.\d+)?(KB|MB|GB)', '', text)
    text = text.replace("\\", "")
    # Add other preprocessing steps as needed
    return text

# Step 1: Preprocess the page_content of each document
preprocessed_docs = []
for doc in converted_docs:
    preprocessed_content = preprocess_text(doc.page_content)
    preprocessed_doc = Document(page_content=preprocessed_content, metadata=doc.metadata)
    preprocessed_docs.append(preprocessed_doc)

In [24]:
for doc in preprocessed_docs:
    doc.metadata["domain"] = "document_info"  # 행정업무 정보 라벨링

In [25]:
#문서 병합
# 2. 문서 병합
all_docs = csv_docs + preprocessed_docs

In [26]:
all_docs

[Document(metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv', 'row': 0, 'domain': 'lecture_info'}, page_content=': 0\n관련 모듈: 프로젝트 수행을 위한 이론 1 : Python\n\n(기초)\n강의 목록: Part1 파이썬 기본기 다지기\n파트명: Ch01. 파이썬에 대한 모든 것\n클립명: Intro. Selena 강좌를 듣기 전 필독! 학습 방법 5단계\n클립 시간: 0:10:52\n수강 일자: 11/14 (목)\n일자별 수강 시간: 4:07:46\n수강 완료 일자: 11/26 (화)'),
 Document(metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv', 'row': 1, 'domain': 'lecture_info'}, page_content=': 1\n관련 모듈: 프로젝트 수행을 위한 이론 1 : Python\n\n(기초)\n강의 목록: Part1 파이썬 기본기 다지기\n파트명: Ch01. 파이썬에 대한 모든 것\n클립명: CH01-01. 파이썬 활용분야\n클립 시간: 0:09:44\n수강 일자: 11/14 (목)\n일자별 수강 시간: 4:07:46\n수강 완료 일자: 11/26 (화)'),
 Document(metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv', 'row': 2, 'domai

# Textspitter 구현

In [27]:
# 1. 텍스트 스플리터 정의
csv_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=500)
html_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=100)

# 2. 문서 타입 분리
csv_docs_only = [doc for doc in all_docs if doc.metadata["domain"] == "lecture_info"]
html_docs_only = [doc for doc in all_docs if doc.metadata["domain"] == "document_info"]

In [ ]:
# 3. 문서 분할
split_csv_docs = csv_splitter.split_documents(csv_docs_only) #lecture
split_html_docs = html_splitter.split_documents(html_docs_only)

# 4. 합치기
split_docs = split_csv_docs + split_html_docs

In [29]:
split_html_docs

[Document(metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/page.txt', 'domain': 'document_info'}, page_content='출석대장양식 / 휴가계획서 작성법\n\n📁\n\n출석대장양식 / 휴가계획서 작성법\n\n제작:\n\n\n\n📁\n\n# 출석대장양식 / 휴가계획서 작성법\n\n휴가계획서 - 다운받아 작성 후 → PDF 파일로 저장 →설문시 출석대장과 함께 설문제출\n\n[패스트캠퍼스강남학원] 휴가계획서'),
 Document(metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/page.txt', 'domain': 'document_info'}, page_content='[패스트캠퍼스강남학원] 휴가계획서\n\n[>](https://forms.gle/zhVtyXNvuxy8BDY5A)\n[휴가제출 폼 링크](https://docs.google.com/forms/d/e/1FAIpQLSd3sIQywt59-md5533dq52sWRubzCE09VdGNWaxtLWANGM0oQ/viewform)\n\n휴가 계획서 작성 방법\n\n\n\n작성자'),
 Document(metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/page.txt', 'domain': 'document_info'}, page_content='휴가 계획서 작성 방법\n\n\n\n작성자\n\n휴가일시\n\n과정명\n\n출석대장 - 모든 출결 정정시 필수로 제출필요\n\n아래 각 과정에 맞는 출석대장 양식 확인해주세요!\n\n다운받아 작성 후 → PDF 파일로 저장 →설문시 출석대장과 함께 설문제출\n👍\n구글폼 제출 후 행정문의 방에 행정 매니저 태깅하여 제출 했다고 남겨주세요\n→ 사유와 일자 공유\n\n👍\n스레드 공유 예시'),
 Document(meta

# 임베딩 모델 정의

In [30]:
from langchain_upstage import UpstageEmbeddings
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")


# 벡터스토어 정의

In [31]:
vectorstore = FAISS.from_documents(documents=split_docs, embedding=embeddings)

In [32]:
# 6. 저장
save_path = "/home/data/Langchain/qa_engine_test/data/vectorstores/lecture_and_docs"
os.makedirs(save_path, exist_ok=True)
vectorstore.save_local(save_path)

print(f"FAISS 벡터스토어 저장 완료: {save_path}")

# 나중에 쓸 때
#vectorstore = FAISS.load_local("vectorstore/course_admin_store", embedding_model)

FAISS 벡터스토어 저장 완료: /home/data/Langchain/qa_engine_test/data/vectorstores/lecture_and_docs


# LCEL 기반 retieval

In [ ]:
# retrievers 미리 구성해두기
retriever_lecture = vectorstore.as_retriever(search_kwargs={"k": 5, "filter": {"domain": "lecture_info"}})
retriever_docu = vectorstore.as_retriever(search_kwargs={"k": 5, "filter": {"domain": "document_info"}})

In [113]:
retriever_lecture.invoke("[Course 1] Part 4. 시스템 프로그래밍 강의는 언제야?")

[Document(id='26089ce6-1735-498c-a4dd-cefe8ba6f0fc', metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv', 'row': 213, 'domain': 'lecture_info'}, page_content=': 213\n관련 모듈: 컴퓨터 공학 개론\n강의 목록: [Course 1] Part 4. 시스템 프로그래밍\n파트명: Ch 1. 오리엔테이션\n클립명: 01. 시스템 프로그래밍의 개념\n클립 시간: 0:06:23\n수강 일자: 12/11 (수)\n일자별 수강 시간: 12:3:47\n수강 완료 일자: 2024. 12. 11'),
 Document(id='104d4132-422f-41c6-9872-e89f97cf54e1', metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv', 'row': 216, 'domain': 'lecture_info'}, page_content=': 216\n관련 모듈: 컴퓨터 공학 개론\n강의 목록: [Course 1] Part 4. 시스템 프로그래밍\n파트명: Ch 2. 파일 다루기\n클립명: 02. 파일 입력\n클립 시간: 0:15:47\n수강 일자: 12/11 (수)\n일자별 수강 시간: 12:3:47\n수강 완료 일자: 2024. 12. 11'),
 Document(id='2089657b-9c2b-43e8-8821-f21a88a56ed2', metadata={'source': '/home/data/Langchain/qa_engine_test/data/files/v2_[커리어 _ Bootc

In [ ]:
'''
llm = HuggingFaceEndpoint(
    endpoint_url= "https://api-inference.huggingface.co/models/upstage/SOLAR-10.7B-Instruct", #"https://api-inference.huggingface.co/models/microsoft/Phi-3-mini-4k-instruct", #, #"https://api-inference.huggingface.co/models/microsoft/Phi-3-mini-4k-instruct" , #"https://api-inference.huggingface.co/models/tiiuae/falcon-7b-instruct",
    huggingfacehub_api_token=hb_api_key,  # 환경 변수에서 불러온 토큰 사용
    temperature=0.1,     
    max_new_tokens = 100,
)
'''

# 템플릿 정의

## domain 구별

In [126]:
chat = ChatUpstage()

In [147]:
domain_prompt = ChatPromptTemplate.from_messages([
    ("system", 
    "다음 사용자 질문을 읽고 도메인을 분류하세요:\n"
    "- lecture_info: 수업, 강의, 일정 관련 질문\n"
    "- document_info: 출석, 휴가, 양식 등 행정 질문\n\n"
    "오직 다음 중 하나만 출력하세요: lecture_info, document_info, None"
    ),
    ("human", "{input}"),  # 사용자 입력을 변수로 사용
    ]
)
    

# 체인을 생성합니다.
domain_chain = (
    domain_prompt
    | chat
    | StrOutputParser()  # 문자열 출력 파서를 사용합니다.
)

In [181]:
print(domain_chain.invoke({"input": "AI 기초 학습 : 딥러닝 모듈에서는 어떤 강의를 배우나요?"}))
print(domain_chain.invoke({"input": "휴가 제출?"}))
print(domain_chain.invoke({"input": "오늘의 날씨는 어떤가요?"}))

강의 계획서를 확인해보니, AI 기초 학습 : 딥러닝 모듈에서는 다음과 같은 강의를 배우게 됩니다:

1. 딥러닝의 개요와 역사
2. 신경망의 구조와 작동 원리
3. 활성화 함수와 손실 함수
4. 역전파 알고리즘
5. 딥러닝 모델의 최적화
6. 딥러닝 프레임워크 소개 (TensorFlow, PyTorch 등)
7. CNN(Convolutional Neural Network)
8. RNN(Recurrent Neural Network)
9. LSTM(Long Short-Term Memory)
10. 딥러닝 모델의 평가와 개선

따라서 사용자 질문의 도메인은 lecture_info입니다.
document_info
None


In [ ]:
'''
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

examples = [
    {"input": "휴가 제출 폼 링크는 무엇인가요?", "output": "https://docs.google.com/forms/d/e/1FAIpQLSd3sIQywt59-md5533dq52sWRubzCE09VdGNWaxtLWANGM0oQ/viewform 입니다."},
    {"input": "Part5. 데이터 분석 및 시각화 프로젝트 강의는 수강일자는 언제인가요?", "output": "수강 시작 일자는 2024. 11. 26 입니다. "},
    {"input": "휴가 제출 양식 작성법은 무엇인가요?", "output": "휴가계획서 - 다운받아 작성 후 → PDF 파일로 저장 →설문시 출석대장과 함께 설문제출 순서대로 진행합니다."},
    
]

# 예제 프롬프트 템플릿 정의
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Few-shot 프롬프트 템플릿 생성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 템플릿 생성
final_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system",
         "**너는 패스트캠퍼스 운영자야.**\n"
         "무조건 답변 시작을 [{domain} info] 시작해.\n"
         "모르면 모른다고 대답해. 답변을 거짓으로 만들지마.\n"
         "사용자가 강의 일정 또는 출석이나 휴가 신청 방법에 대해 묻는 질문에 정확하게 대답해.\n"
         "기존 정보에 없으면 없다고 대답해.\n\n"

         "## **질문 분류 기준**\n"
         "1. **강의 일정 질문**\n"
         "- 사용자가 여러 수업의 관련 모듈에 따른 강의 목록과 강의 수강 일정을 요청하는 질문\n"
         "- 예시: ' AI 기초 학습 : 딥러닝 모듈에서는 어떤 강의를 배우나요?', 'Machine Learning Basic 강의 수강 일자를 알려줘'\n\n"

         "2. **출석 관련 질문**\n"
         "- 예시: '출석 인정 제출은 어떻게 하나요?', '지각하면 어떻게 되나요?'\n\n"

         "3. **휴가 신청 관련 질문**\n"
         "- 예시: '휴가 폼 어디 있어요?', '휴가 신청은 며칠 전에 해야 하나요?'\n\n"
         "**질문을 정확히 이해하고 관련된 답변만 하세요.**\n"

         "## **[응답 형식 가이드]**\n"
         "- 강의 일정 관련: '수강 시작 일자는 YYYY. MM. DD 입니다.'\n"
         "- 출석 관련: '출석 인정 제출은 ~입니다.'\n"
         "- 휴가 관련: '휴가 신청은 ~입니다.'\n\n"
         "형식에서 벗어나지 마세요. 없으면 '정보를 찾을 수 없습니다'라고 대답하세요.\n"
         "모르면 모른다고 솔직히 말하세요. 거짓으로 대답하지 마세요.\n"
),
few_shot_prompt,
("human", "{input}"),  # 사용자 입력을 변수로 사용
    ]
)
'''


## domain별 chain 생성

In [201]:
from langchain.chains import RetrievalQA
lecture_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",  # 간단한 기본 체인
    retriever=vectorstore.as_retriever(
        search_kwargs={
            "k": 5,
            "filter": {"domain": "lecture_info"}  # 강의 관련 도메인만 검색
        }
    ),
)
document_chain = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(
        search_kwargs={
            "k": 5,
            "filter": {"domain": "document_info"}  # 출석/휴가 관련만 검색
        }
    ),
)

# 일반 질문용 프롬프트
general_chain = (
    PromptTemplate.from_template(
        "사용자의 일반적인 질문에 답변해주세요.\n\n"
        "질문: {input}\n"
        "답변: "

    ) | chat
)


## route

In [202]:
def route(info):
    domain = info["domain"].strip().lower()
    if "lecture" in domain:
        return lecture_chain
    elif "document" in domain:
        return document_chain
    else:
        return RunnableLambda(lambda x: {"result": general_chain.invoke({"input": x["query"]})})

# full chain

In [200]:
from langchain.schema.runnable import RunnableMap

# 전체 체인
full_chain = RunnableMap({
    "domain": domain_chain,
    "query": lambda x: x["input"]  # query로 key 맞춰줌
}) | RunnableLambda(route)

# test

In [211]:
# 전체 체인
#full_chain = {"domain": domain_chain, "input": lambda x: x["input"]} | RunnableLambda(route)

# 질문
question1 = "[Course 1] Part 4. 시스템 프로그래밍 강의 관련 모듈은?"
question2 = "휴가 제출 폼 링크는 어디인가요?"
question3 = "저녁 뭐먹지?"
response = full_chain.invoke({"input": question1})
print(f" 답변 : {response['result']}")

 답변 : [Course 1] Part 4. 시스템 프로그래밍 강의의 관련 모듈은 컴퓨터 공학 개론입니다.


In [ ]:
'''
def format_response(domain, question, answer):
    return f"""🧠 *도메인*: `{domain}`
❓ *질문*: {question}
💬 *답변*: {answer}

question2 = "휴가 제출 폼 링크는 어디인가요?"
response = full_chain.invoke({"input": question2})
formatted = format_response(response["domain"], question2, response["result"])
print(formatted)
"""

In [ ]:
file_path = os.path.join(os.path.dirname(os.getcwd()), 
                                          'data/files/[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv')
lecture_df = pd.read_csv(file_path, skiprows = 4)
print(lecture_df.head())  # 데이터 미리보기
print(lecture_df.columns)  # 컬럼 확인

lecture_df = lecture_df.drop(['Unnamed: 0'], axis = 1)

# 전처리함수 / 병합 셀 fill
def fill_between_non_nan(df, column):
    filled_values = []
    temp_value = None  # 마지막으로 본 NaN이 아닌 값 저장
    
    for value in df[column]:
        if pd.notna(value):  # NaN이 아닌 값이 나타나면 저장
            temp_value = value
        filled_values.append(temp_value)  # 현재 값 또는 가장 최근 값을 추가
    
    df[column] = filled_values  # 결과 반영
    
    return df

# csv
for i in lecture_df.columns:
    lecture_df2 = fill_between_non_nan(lecture_df, i)

lecture_df2.to_csv(os.path.join(os.path.dirname(os.getcwd()), 
                               'data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv'))
# skiprows = 4 적용한 CSV 로더 생성
loader = CSVLoader(
    file_path=os.path.join(os.path.dirname(os.getcwd()),
                           'data/files/v2_[커리어 _ Bootcamp] Upstage AI Lab_6기_수강생 공유용 커리큘럼 - 전체 온라인 강의 리스트(필수).csv'),
                           csv_args={
                               'delimiter': ',',
                               }
                               )

# 데이터 로드
csv_docs = loader.load()